## No use of class

In [ ]:
# DATA - no labels

import os
from PIL import Image
import glob

transform = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.ToTensor(),             # convert to [C,H,W] tensor
    transforms.Normalize((0.5,), (0.5,))  # scale to [-1,1]
])

PATCHES_PATH = "/Volumes/tereju_disk/RECETOX/autoencoder/coad/kather01/01_TUMOR"

# Custom Dataset for WSI patches
class WSIPatchDataset(Dataset):
    def __init__(self, patches_dir, transform=None):
        self.patches_dir = patches_dir
        self.transform = transform
        
        # Find all .tif files in the directory
        self.patch_files = glob.glob(os.path.join(patches_dir, "*.tif"))
        print(f"Found {len(self.patch_files)} .tif files")
        
        if len(self.patch_files) == 0:
            # Try other common extensions
            for ext in ["*.jpg", "*.jpeg", "*.png", "*.tiff"]:
                self.patch_files.extend(glob.glob(os.path.join(patches_dir, ext)))
        
        print(f"Total patches found: {len(self.patch_files)}")
        
        if len(self.patch_files) == 0:
            raise FileNotFoundError(f"No image files found in {patches_dir}")
    
    def __len__(self):
        return len(self.patch_files)
    
    def __getitem__(self, idx):
        patch_path = self.patch_files[idx]
        
        # Load image
        image = Image.open(patch_path).convert('RGB')
        
        # Apply transforms
        if self.transform:
            image = self.transform(image)
        
        return image, 0  # Return 0 as dummy label for autoencoder

# Create dataset and dataloader
try:
    dataset = WSIPatchDataset(PATCHES_PATH, transform=transform)
    dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
    print(f"Dataset created successfully with {len(dataset)} patches")
except Exception as e:
    print(f"Error creating dataset: {e}")
    print(f"Please check if the path exists: {PATCHES_PATH}")

In [ ]:
# Autoencoder Architecture - NO Class

# Encoder
encoder = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3, stride=2, padding=1),                               # [B, 32, 75, 75]
    nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1),                              # [B, 64, 38, 38]
    nn.ReLU(),
    nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),                             # [B, 128, 19, 19]
    nn.ReLU()
)

# Decoder
decoder = nn.Sequential(
    nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, output_padding=1),  # [B, 64, 38, 38]
    nn.ReLU(),
    nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, output_padding=1),   # [B, 32, 75, 75]
    nn.ReLU(),
    nn.ConvTranspose2d(32, 3, kernel_size=3, stride=2, padding=1, output_padding=1),    # [B, 3, 150, 150]
    nn.Sigmoid()
)

# MODEL: apply encoder then decoder
def autoencoder(x):
    z = encoder(x)
    out = decoder(z)
    return out

# Example: run a forward pass
x = torch.randn(1, 3, 150, 150)   # fake input
y = autoencoder(x)
print(y.shape)  # should be [1,3,150,150]